In [2]:
def find_lr(dataloader, model, optimizer, criterion, init_value = 1e-8, final_value=10., beta = 0.98):
    from torch import Tensor
    import math
    
    # Save current state_dict of model
    original_state = model.state_dict
    
    num = len(dataloader)-1
    mult = (final_value / init_value) ** (1/num)
    lr = init_value
    opt.param_groups[0]['lr'] = lr
    avg_loss = 0.
    best_loss = 0.
    batch_num = 0
    losses = []
    log_lrs = []
    for data in dataloader:
        batch_num += 1
        #As before, get the loss for this mini-batch of inputs/outputs
        x_cats, x_conts, y = data
        x_cats, x_conts, y = torch.LongTensor(x_cats), torch.FloatTensor(x_conts), torch.FloatTensor(y)
        x_cats.requires_grad, x_conts.requires_grad = False, False
        optimizer.zero_grad()
        outputs = model(x_cats, x_conts)
        loss = criterion(outputs, y)
        #Compute the smoothed loss
        print(loss)
        avg_loss = beta * avg_loss + (1-beta) * loss.data
        smoothed_loss = avg_loss / (1 - beta**batch_num)
        #Stop if the loss is exploding
        if batch_num > 1 and smoothed_loss > 4 * best_loss:
            return log_lrs, losses
        #Record the best loss
        if smoothed_loss < best_loss or batch_num==1:
            best_loss = smoothed_loss
            print('best loss: {}'.format(best_loss))
        #Store the values
        losses.append(smoothed_loss)
        log_lrs.append(math.log10(lr))
#         log_lrs.append(lr)
        #Do the SGD step
        loss.backward()
        optimizer.step()
        #Update the lr for the next step
        lr *= mult
        optimizer.param_groups[0]['lr'] = lr
        
    # Reset model state_dict
    model.state_dict = original_state
    
    return log_lrs, losses

lrs, losses = find_lr(model_data.trn_dl, emb_model, opt, crit)